In [68]:
from qiskit import *
from qiskit.visualization import *
import numpy as np

simulator = BasicAer.get_backend("qasm_simulator")

In [69]:
def single_exp_val(var_params, pauli_op):

    qc = QuantumCircuit(2,1)

    qc.h(0)

    qc.rx(var_params[0],1)
    qc.ry(var_params[1],1)
    qc.rz(var_params[2],1)

    qc.rx(var_params[3],1)
    qc.ry(var_params[4],1)
    qc.rz(var_params[5],1)

    if pauli_op == 'Z':
        qc.cz(0,1)
    elif pauli_op == 'X':
        qc.cx(0,1)

    qc.h(0)
    qc.measure(0,0)

    num_shots = 2048

    counts = execute(qc, backend=simulator, shots=num_shots).result().get_counts(qc)
    
    if '0' not in counts:
        counts['0']=0
    if '1' not in counts:
        counts['1']=0

    exp_val = (counts['0'] - counts['1'])/num_shots
        
    return exp_val

In [70]:
3*single_exp_val([0,1,2,3,4,5],'Z') - 2*single_exp_val([0,1,2,3,4,5],'X') + 1

2.3388671875

In [71]:
z = np.matrix([[1,0],[0,-1]])
i = np.identity(2)
x = np.matrix([[0,1],[1,0]])

matrix_h = 3*z - 2*x + i

In [72]:
def cost_function(params):
    cost = 3*single_exp_val(params,'Z') - 2*single_exp_val(params,'X') + 1
    return cost

In [73]:
cost_function([0,1,2,3,4,5])

2.5029296875

In [74]:
import scipy
from scipy import optimize
res = scipy.optimize.minimize(cost_function, x0=np.ones(6), 
                                method = 'COBYLA', callback=None,
                                options={'maxiter': 1000, 'ftol': 1e-06, 'iprint': 1, 'disp': True, 
                                'eps': 1.4901161193847656e-08, 'finite_diff_rel_step': None})

In [75]:
res

     fun: -2.4853515625
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 52
  status: 1
 success: True
       x: array([1.00001465, 1.00004545, 1.00003041, 2.00001023, 0.99992012,
       1.00001757])

In [76]:
cost_function(res['x'])

-2.517578125

In [77]:
print(min(np.linalg.eig(matrix_h)[0]))

-2.6055512754639896
